In [1]:
import pandas as pd
import numpy as np
import datetime
import seaborn as sns
sns.set_style('whitegrid')
from os import listdir
from os.path import isfile, join
from pandas_datareader import data
import yfinance as yf
from pandas_datareader import wb

import quandl
import pandas_datareader as pdr
import numpy as np

from datetime import date

import matplotlib.pyplot as plt
import matplotlib.ticker as mtick

import copy
from tqdm import tqdm

import math
# import pmdarima as pm
yf.pdr_override()

In [4]:
start_date = "2015-01-01"
end_date = "2022-12-31"

stock_name = 'AAPL'
to_pred_stock_df = data.get_data_yahoo(tickers=stock_name, start = start_date, end = end_date)
to_pred_stock_df = to_pred_stock_df.drop(to_pred_stock_df[to_pred_stock_df.index<start_date].index)
to_pred_stock_df.reset_index(inplace=True)

general_information = ['^DJI', '^IXIC', '^GSPC', '^RLG', '^NYA', '^VIX', 'DX-Y.NYB', 'EURUSD=X', 'GBPUSD=X', 'GC=F', 'SI=F', 'CL=F']
general_df = []

for index, the_name in enumerate(general_information):
    stock_df = data.get_data_yahoo(tickers=the_name, start=start_date, end=end_date)
    stock_df.reset_index(inplace=True)
    print(the_name)
    print(stock_df.head())
    if index == 0:
        general_df = to_pred_stock_df[["Date", "Close", 'Volume', 'Adj Close']]
        general_df = pd.merge(general_df, stock_df[["Date", "Close"]].rename(columns={"Close": str(the_name)+"_Close"}), on="Date")
    else:
        general_df = pd.merge(general_df, stock_df[["Date", "Close"]].rename(columns={"Close": str(the_name)+"_Close"}), on="Date")


# state_var = {'inflation':'FP.CPI.TOTL.ZG', 'GDP':'NY.GDP.PCAP.KD', 'GDPpercapita':'NY.GDP.PCAP.CD', 'Grossfixedcapital':'NE.GDI.FTOT.CD','CO2':'SP.POP.TOTL', \
#              'Stockstradedpergdp':'CM.MKT.TRAD.GD.ZS'}
# selected_countries=['US', 'CA', 'MX']
# print(general_df.head())
# for the_key in state_var:
#     print(state_var[the_key])
#     the_df = wb.download(indicator=state_var[the_key], country=selected_countries, start=start_date, end=end_date)
#     print(the_df.head())
#     # general_df = pd.merge(general_df, stock_df[["Date", "Close"]].rename(columns={"Close": str(the_name) + "_Close"}),
#     #                       on="Date")

report_list = [
  'WM1NS', # M1 Supply
  'WM2NS', # M2 Supply
  'ICSA', # Unemployment
  'CCSA', # Continued Unemployment
  'JTSJOL', # Job Openings: Total Nonfarm
  'PAYEMS', # Non-Farm Employment
  'RSXFS', # Retail Sales
  'TCU', # Capacity Utilization
  'UMCSENT', # Consumer Sentiment Index
  'BUSINV', # Business Inventories
  'INDPRO', # Industrial Production Index
  'GACDFSA066MSFRBPHI', # Philidelphia Fed Manufacturing Index
  'GACDISA066MSFRBNY', # Empire State Manufacturing Index
  'BACTSAMFRBDAL', # Current General Business Activity; Diffusion Index for Texas
  'IR', # Import Price Index
  'IQ', # Export Price Index
  'PPIACO', # Producer Price Index - all
  'CPIAUCSL', # Consumer Price Index - all
  'CPILFESL', # Consumer Price Index (Core)
  'MICH', # University of Michigan: Inflation Expectation
  'CSCICP03USM665S', # Consumer Opinion Surveys: Confidence Indicators: Composite Indicators: OECD Indicator for the United States
]
macro_indicators = dict()
tq_fred = tqdm(report_list)
tq_fred.set_description('Downloading stats from FRED:')
idx = pd.date_range(start_date, end_date)
print(idx)
for indicator in tq_fred:
  # tq_fred.set_description(indicator)
  # macro_indicators[indicator] = pdr.DataReader(indicator, "fred", start=start, timeout=90)
  macro_indicators[indicator] = pdr.fred.FredReader(indicator, start=start_date, end = end_date, timeout=90).read()
  print(macro_indicators[indicator].index)
  macro_indicators[indicator] = macro_indicators[indicator].reindex(idx, fill_value=0).reset_index()
  print(macro_indicators[indicator])
  print(macro_indicators[indicator].columns.values)
  macro_indicators[indicator] = macro_indicators[indicator].rename(columns={"index": "Date"})
  macro_indicators[indicator][indicator] = macro_indicators[indicator][indicator].replace(to_replace=0, method='ffill')
  general_df = pd.merge(general_df, macro_indicators[indicator], how="left", on=['Date'])
  # print(macro_indicators[indicator])
general_df.to_csv('generated_df.csv')
print(general_df)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
^DJI
        Date          Open          High           Low         Close  \
0 2014-12-31  17987.660156  18043.220703  17820.880859  17823.070312   
1 2015-01-02  17823.070312  17951.779297  17731.300781  17832.990234   
2 2015-01-05  17821.300781  17821.300781  17475.929688  17501.650391   
3 2015-01-06  17504.179688  17581.050781  17262.369141  17371.640625   
4 2015-01-07  17374.779297  17597.080078  17374.779297  17584.519531   

      Adj Close     Volume  
0  17823.070312   82840000  
1  17832.990234   76270000  
2  17501.650391  116160000  
3  17371.640625  101870000  
4  17584.519531   91030000  
[*********************100%***********************]  1 of 1 completed
^IXIC
        Date         Open         High          Low        Close    Adj Close  \
0 2014-12-31  4790.529785  4806.430176  4734.109863  4736.049805  4736.049805   
1 2015-01-02 

DatetimeIndex(['2015-01-01', '2015-01-02', '2015-01-03', '2015-01-04',
               '2015-01-05', '2015-01-06', '2015-01-07', '2015-01-08',
               '2015-01-09', '2015-01-10',
               ...
               '2022-12-22', '2022-12-23', '2022-12-24', '2022-12-25',
               '2022-12-26', '2022-12-27', '2022-12-28', '2022-12-29',
               '2022-12-30', '2022-12-31'],
              dtype='datetime64[ns]', length=2922, freq='D')


DatetimeIndex(['2015-01-05', '2015-01-12', '2015-01-19', '2015-01-26',
               '2015-02-02', '2015-02-09', '2015-02-16', '2015-02-23',
               '2015-03-02', '2015-03-09',
               ...
               '2022-10-24', '2022-10-31', '2022-11-07', '2022-11-14',
               '2022-11-21', '2022-11-28', '2022-12-05', '2022-12-12',
               '2022-12-19', '2022-12-26'],
              dtype='datetime64[ns]', name='DATE', length=417, freq=None)
          index   WM1NS
0    2015-01-01     0.0
1    2015-01-02     0.0
2    2015-01-03     0.0
3    2015-01-04     0.0
4    2015-01-05  3030.2
...         ...     ...
2917 2022-12-27     0.0
2918 2022-12-28     0.0
2919 2022-12-29     0.0
2920 2022-12-30     0.0
2921 2022-12-31     0.0

[2922 rows x 2 columns]
['index' 'WM1NS']


DatetimeIndex(['2015-01-05', '2015-01-12', '2015-01-19', '2015-01-26',
               '2015-02-02', '2015-02-09', '2015-02-16', '2015-02-23',
               '2015-03-02', '2015-03-09',
               ...
               '2022-10-24', '2022-10-31', '2022-11-07', '2022-11-14',
               '2022-11-21', '2022-11-28', '2022-12-05', '2022-12-12',
               '2022-12-19', '2022-12-26'],
              dtype='datetime64[ns]', name='DATE', length=417, freq=None)
          index    WM2NS
0    2015-01-01      0.0
1    2015-01-02      0.0
2    2015-01-03      0.0
3    2015-01-04      0.0
4    2015-01-05  11820.4
...         ...      ...
2917 2022-12-27      0.0
2918 2022-12-28      0.0
2919 2022-12-29      0.0
2920 2022-12-30      0.0
2921 2022-12-31      0.0

[2922 rows x 2 columns]
['index' 'WM2NS']


DatetimeIndex(['2015-01-03', '2015-01-10', '2015-01-17', '2015-01-24',
               '2015-01-31', '2015-02-07', '2015-02-14', '2015-02-21',
               '2015-02-28', '2015-03-07',
               ...
               '2022-10-29', '2022-11-05', '2022-11-12', '2022-11-19',
               '2022-11-26', '2022-12-03', '2022-12-10', '2022-12-17',
               '2022-12-24', '2022-12-31'],
              dtype='datetime64[ns]', name='DATE', length=418, freq=None)
          index    ICSA
0    2015-01-01       0
1    2015-01-02       0
2    2015-01-03  294000
3    2015-01-04       0
4    2015-01-05       0
...         ...     ...
2917 2022-12-27       0
2918 2022-12-28       0
2919 2022-12-29       0
2920 2022-12-30       0
2921 2022-12-31  206000

[2922 rows x 2 columns]
['index' 'ICSA']


...

DatetimeIndex(['2015-01-01', '2015-02-01', '2015-03-01', '2015-04-01',
               '2015-05-01', '2015-06-01', '2015-07-01', '2015-08-01',
               '2015-09-01', '2015-10-01', '2015-11-01', '2015-12-01',
               '2016-01-01', '2016-02-01', '2016-03-01', '2016-04-01',
               '2016-05-01', '2016-06-01', '2016-07-01', '2016-08-01',
               '2016-09-01', '2016-10-01', '2016-11-01', '2016-12-01',
               '2017-01-01', '2017-02-01', '2017-03-01', '2017-04-01',
               '2017-05-01', '2017-06-01', '2017-07-01', '2017-08-01',
               '2017-09-01', '2017-10-01', '2017-11-01', '2017-12-01',
               '2018-01-01', '2018-02-01', '2018-03-01', '2018-04-01',
               '2018-05-01', '2018-06-01', '2018-07-01', '2018-08-01',
               '2018-09-01', '2018-10-01', '2018-11-01', '2018-12-01',
               '2019-01-01', '2019-02-01', '2019-03-01', '2019-04-01',
               '2019-05-01', '2019-06-01', '2019-07-01', '2019-08-01',
      

DatetimeIndex(['2015-01-01', '2015-02-01', '2015-03-01', '2015-04-01',
               '2015-05-01', '2015-06-01', '2015-07-01', '2015-08-01',
               '2015-09-01', '2015-10-01', '2015-11-01', '2015-12-01',
               '2016-01-01', '2016-02-01', '2016-03-01', '2016-04-01',
               '2016-05-01', '2016-06-01', '2016-07-01', '2016-08-01',
               '2016-09-01', '2016-10-01', '2016-11-01', '2016-12-01',
               '2017-01-01', '2017-02-01', '2017-03-01', '2017-04-01',
               '2017-05-01', '2017-06-01', '2017-07-01', '2017-08-01',
               '2017-09-01', '2017-10-01', '2017-11-01', '2017-12-01',
               '2018-01-01', '2018-02-01', '2018-03-01', '2018-04-01',
               '2018-05-01', '2018-06-01', '2018-07-01', '2018-08-01',
               '2018-09-01', '2018-10-01', '2018-11-01', '2018-12-01',
               '2019-01-01', '2019-02-01', '2019-03-01', '2019-04-01',
               '2019-05-01', '2019-06-01', '2019-07-01', '2019-08-01',
      

DatetimeIndex(['2015-01-01', '2015-02-01', '2015-03-01', '2015-04-01',
               '2015-05-01', '2015-06-01', '2015-07-01', '2015-08-01',
               '2015-09-01', '2015-10-01', '2015-11-01', '2015-12-01',
               '2016-01-01', '2016-02-01', '2016-03-01', '2016-04-01',
               '2016-05-01', '2016-06-01', '2016-07-01', '2016-08-01',
               '2016-09-01', '2016-10-01', '2016-11-01', '2016-12-01',
               '2017-01-01', '2017-02-01', '2017-03-01', '2017-04-01',
               '2017-05-01', '2017-06-01', '2017-07-01', '2017-08-01',
               '2017-09-01', '2017-10-01', '2017-11-01', '2017-12-01',
               '2018-01-01', '2018-02-01', '2018-03-01', '2018-04-01',
               '2018-05-01', '2018-06-01', '2018-07-01', '2018-08-01',
               '2018-09-01', '2018-10-01', '2018-11-01', '2018-12-01',
               '2019-01-01', '2019-02-01', '2019-03-01', '2019-04-01',
               '2019-05-01', '2019-06-01', '2019-07-01', '2019-08-01',
      

DatetimeIndex(['2015-01-01', '2015-02-01', '2015-03-01', '2015-04-01',
               '2015-05-01', '2015-06-01', '2015-07-01', '2015-08-01',
               '2015-09-01', '2015-10-01', '2015-11-01', '2015-12-01',
               '2016-01-01', '2016-02-01', '2016-03-01', '2016-04-01',
               '2016-05-01', '2016-06-01', '2016-07-01', '2016-08-01',
               '2016-09-01', '2016-10-01', '2016-11-01', '2016-12-01',
               '2017-01-01', '2017-02-01', '2017-03-01', '2017-04-01',
               '2017-05-01', '2017-06-01', '2017-07-01', '2017-08-01',
               '2017-09-01', '2017-10-01', '2017-11-01', '2017-12-01',
               '2018-01-01', '2018-02-01', '2018-03-01', '2018-04-01',
               '2018-05-01', '2018-06-01', '2018-07-01', '2018-08-01',
               '2018-09-01', '2018-10-01', '2018-11-01', '2018-12-01',
               '2019-01-01', '2019-02-01', '2019-03-01', '2019-04-01',
               '2019-05-01', '2019-06-01', '2019-07-01', '2019-08-01',
      

DatetimeIndex(['2015-01-01', '2015-02-01', '2015-03-01', '2015-04-01',
               '2015-05-01', '2015-06-01', '2015-07-01', '2015-08-01',
               '2015-09-01', '2015-10-01', '2015-11-01', '2015-12-01',
               '2016-01-01', '2016-02-01', '2016-03-01', '2016-04-01',
               '2016-05-01', '2016-06-01', '2016-07-01', '2016-08-01',
               '2016-09-01', '2016-10-01', '2016-11-01', '2016-12-01',
               '2017-01-01', '2017-02-01', '2017-03-01', '2017-04-01',
               '2017-05-01', '2017-06-01', '2017-07-01', '2017-08-01',
               '2017-09-01', '2017-10-01', '2017-11-01', '2017-12-01',
               '2018-01-01', '2018-02-01', '2018-03-01', '2018-04-01',
               '2018-05-01', '2018-06-01', '2018-07-01', '2018-08-01',
               '2018-09-01', '2018-10-01', '2018-11-01', '2018-12-01',
               '2019-01-01', '2019-02-01', '2019-03-01', '2019-04-01',
               '2019-05-01', '2019-06-01', '2019-07-01', '2019-08-01',
      

In [3]:
to_pred_stock_df = data.get_data_yahoo(tickers=stock_name, start = '2015-01-02', end = '2022-12-30')
to_pred_stock_df

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2015-01-02,27.847500,27.860001,26.837500,27.332500,24.565697,212818400
2015-01-05,27.072500,27.162500,26.352501,26.562500,23.873642,257142000
2015-01-06,26.635000,26.857500,26.157499,26.565001,23.875887,263188400
2015-01-07,26.799999,27.049999,26.674999,26.937500,24.210678,160423600
2015-01-08,27.307501,28.037500,27.174999,27.972500,25.140915,237458000
...,...,...,...,...,...,...
2022-12-22,134.350006,134.559998,130.300003,132.229996,132.028412,77852100
2022-12-23,130.919998,132.419998,129.639999,131.860001,131.658981,63814900
2022-12-27,131.380005,131.410004,128.720001,130.029999,129.831772,69007800
